<a href="https://colab.research.google.com/github/wangyao049/Keras-Tutorial/blob/master/Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 用函数API实现双输入问答系统

In [0]:
import keras
from keras.models import Model
from keras import layers
from keras import Input

In [0]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)

encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')

embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)

encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size,size=(num_samples, max_length))

answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)
model.fit([text, question], answers, epochs=10, batch_size=128)
model.fit({'text': text, 'question': question}, answers,
epochs=10, batch_size=128)

## 2. 层组成的有向无环图
Inception模块



In [0]:
from keras.layers import Input
from keras import layers

x = Input(shape=(128,128,1), dtype='float32')

branch_a = layers.Conv2D(128, 1, activation='relu', strides=2, padding='same')(x)

branch_b = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2, padding='same')(x)
branch_c = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)



残差网络（residual connection）

In [0]:
# 恒等残差连接(identity residual connection)
from keras import layers

x = Input(shape=(128, 128, 1), dtype='float32')
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
# 将原始x与输入特征相加
y = layers.add([y, x])

print(y.shape)

# 如果特征图的尺寸不同，实现残差连接的方法如下，用的是线性残差连接(linear residual connection)

x = Input(shape=(128, 128, 1), dtype='float32')
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])

print(x.shape)
print(y.shape)

## 3. 使用Keras回调函数来监控深度学习模型

In [0]:
# ModelCheckpoint 与 EarlyStopping回调函数 
# EarlyStopping: 如果监控目标在指定轮数内不再改善，则中断模型（例如，在刚开始过拟合的时候就中断训练，从而避免用更少的轮次重新训练模型。）
# ModelCheckpoint： 可以在训练中不断的保存模型权重（也可以选择只保存目前的最佳模型，即一轮结束后具有最佳性能的模型）


callbacks_list = [
    keras.callbacks.EarlyStopping(monitor = 'acc', patience=1), 
    keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True,)]

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.fit(x, y, batch_size=32, callbacks=callbacks_list, validation_data=(x_val, y_val))

In [0]:
# ReduceLROnPlateau回调函数： 如果验证损失不再改善，可以使用这个回调函数来降低学习率。

callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10,)]

In [0]:
import matplotlib.pyplot as plt
% matplotlib inline
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [0]:
plot_model(model, show_shapes=True, to_file='model.png')


In [0]:
from PIL import Image
img = Image.open('model.png')
plt.figure('model')
plt.imshow(img)
plt.show()